Import PySpark & Create Spark Session

In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("Week3_SupplyChain").getOrCreate()



In [9]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_orders.csv to cleaned_orders.csv


Load the CSV into PySpark DataFrame

In [10]:
orders_df = spark.read.csv("cleaned_orders.csv", header=True, inferSchema=True)
orders_df.show(5)


+--------+-----------+-------+----------+-------------+----------+----------+
|order_id|supplier_id|item_id|order_date|delivery_date|delay_days|is_delayed|
+--------+-----------+-------+----------+-------------+----------+----------+
|    1001|       S001|   I001|2024-07-01|   2024-07-03|       392|         1|
|    1002|       S002|   I002|2024-07-02|   2024-07-08|       387|         1|
|    1003|       S001|   I003|2024-07-03|   2024-07-06|       389|         1|
|    1004|       S003|   I004|2024-07-05|   2024-07-10|       385|         1|
|    1005|       S002|   I005|2024-07-06|   2024-07-09|       386|         1|
+--------+-----------+-------+----------+-------------+----------+----------+
only showing top 5 rows



Filter Delayed Shipments

In [11]:
delayed_orders = orders_df.filter(col("is_delayed") == 1)
delayed_orders.show(5)


+--------+-----------+-------+----------+-------------+----------+----------+
|order_id|supplier_id|item_id|order_date|delivery_date|delay_days|is_delayed|
+--------+-----------+-------+----------+-------------+----------+----------+
|    1001|       S001|   I001|2024-07-01|   2024-07-03|       392|         1|
|    1002|       S002|   I002|2024-07-02|   2024-07-08|       387|         1|
|    1003|       S001|   I003|2024-07-03|   2024-07-06|       389|         1|
|    1004|       S003|   I004|2024-07-05|   2024-07-10|       385|         1|
|    1005|       S002|   I005|2024-07-06|   2024-07-09|       386|         1|
+--------+-----------+-------+----------+-------------+----------+----------+
only showing top 5 rows



Group by Supplier and Count Delays

In [12]:
grouped = delayed_orders.groupBy("supplier_id").count()
grouped = grouped.withColumnRenamed("count", "delayed_order_count")
grouped.show()


+-----------+-------------------+
|supplier_id|delayed_order_count|
+-----------+-------------------+
|       S001|                  7|
|       S002|                  7|
|       S003|                  6|
+-----------+-------------------+



In [15]:
grouped.orderBy("delayed_order_count", ascending=False)


DataFrame[supplier_id: string, delayed_order_count: bigint]

In [16]:
grouped.write.csv("delayed_by_supplier", header=True, mode="overwrite")
